In [ ]:
import pyspark
conf = pyspark.SparkConf()
conf.set('spark.jars.packages', 
         "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1,com.databricks:spark-xml_2.12:0.18.0")
conf.set('spark.driver.memory','8g')
sc = pyspark.SparkContext(conf=conf)
spark = pyspark.SQLContext.getOrCreate(sc)
spark

In [ ]:
spark.sparkSession

In [ ]:
from pymongo import MongoClient       

client = MongoClient('10.32.32.4')

In [ ]:
client.server_info()

In [ ]:
db=client.test
db

In [ ]:
# Get Scala Version
sc._gateway.jvm.scala.util.Properties.versionString()

In [ ]:
import os
from zipfile import ZipFile
source_path = "/scratch/work/public/proquest/proquest_hnp/BostonGlobe/BG_20151210212722_00001.zip"
with ZipFile(source_path, "r") as zip:
    print(zip.infolist())
    zip.extract('250949924.xml', 'zip_tmp')

In [ ]:
df = spark.read\
    .option('rootTag', 'Record')\
    .option('rowTag', 'Record')\
    .format("xml").load("zip_tmp/250949924.xml")

In [ ]:
df.printSchema()

In [ ]:
df.select('RecordID', 'Publisher', 'AlphaPubDate', 'NumericPubDate', 'RecordTitle', 'SourceType').show(2)

In [ ]:
# Save to mongo
df.write.format('com.mongodb.spark.sql.DefaultSource')\
    .option('uri','mongodb://10.32.32.4/test.newspapers')\
    .save()

In [ ]:
db.list_collection_names()

In [ ]:
db.newspapers.find_one()